# Python com SQL

- Muitos aplicativos interagem com dados. 
- Algumas linguagens de programação já vem com modulos para usar essas interações.
- Outras linguagens usam uma terceiro pacote, sendo necessario a intalação. 
- Nesta aula vamos mostrar as diferentes bibliotecas de Python SQL.
- Vamos desenvolver uma pequena database para uma aplicação de social media. A database vai consistir de:
    - Usuarios;
    - Posts;
    - Comentarios;;
    - Likes.

### Usando Python SQL para conectar com uma database

- Antes de interagir com qualquer database através de Python, é necessario se conectar com a database. 
- É necessario os servidores dos MySQL e PostgreSQL ligados e rodando antes de executar códigos.

### MySQL

- Não existe modulo padrão SQL para python;
- É necessario a instalação.
    - conda install -c anaconda mysql-connector-python, ou
    - pip install mysql-connector-python
- MySQL é um sistema de gerenciamento de banco de dados baseado em servidor. 
- Um servidor MySQL pode ter vários bancos de dados. Ao contrário do SQLite, em que criar uma conexão equivale a criar um banco de dados, um banco de dados MySQL possui um processo de duas etapas para a criação do banco de dados:

    - Faça uma conexão com um servidor MySQL.
    - Execute uma consulta separada para criar o banco de dados.

In [141]:
import mysql.connector
from mysql.connector import Error

def create_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("localhost", "root", "2303")

Connection to MySQL DB successful


- No script acima, você define uma função create_connection()que aceita três parâmetros:

    - nome de anfitrião
    - nome do usuário
    - senha do usuário
- O módulo mysql.connector contém um método .connect() que você usa na linha 7 para conectar-se a um servidor de banco de dados MySQL. Depois que a conexão é estabelecida, o objeto connection é retornado à função de chamada. - - - Finalmente, na linha 18 você liga create_connection()com o nome do host, nome de usuário e senha.

- Até agora, você apenas estabeleceu a conexão. O banco de dados ainda não foi criado. Para fazer isso, você definirá outra função create_database()que aceita dois parâmetros:

    - connection: é o objeto connectiono para o servidor de banco de dados com o qual você deseja interagir.
    - query: é a consulta que cria o banco de dados.

In [142]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
        #Crie um banco de dados nomeado sm_app para seu aplicativo de mídia social 
        #no servidor de banco de dados MySQL:
        
create_database_query = "CREATE DATABASE sm_app"
create_database(connection, create_database_query)

Database created successfully


- Agora você criou um banco sm_appde dados no servidor de banco de dados. No entanto, o objeto connection retornado pelo create_connection() é conectado ao servidor de banco de dados MySQL. 
- Você precisa se conectar ao sm_app banco de dados. Para fazer isso, você pode modificar create_connection() da seguinte maneira:

In [143]:
def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

- Você pode ver na linha 8 que create_connection()agora aceita um parâmetro adicional chamado db_name. Este parâmetro especifica o nome do banco de dados ao qual você deseja se conectar. Você pode passar o nome do banco de dados ao qual deseja se conectar ao chamar esta função:

In [144]:
connection = create_connection("localhost", "root", "2303", "sm_app")
#O script acima chama create_connection() e conecta com sucesso ao sm_appbanco de dados.

Connection to MySQL DB successful


### Criando Tabelas

- Você usará o módulo Python SQL mysql-connector-python para criar tabelas no MySQL. 
- Assim como no SQLite, você precisa passar sua consulta para cursor.execute(), retornada chamando .cursor() o objeto connection. 
- Você pode criar outra função execute_query() que aceite a string connectione query:

In [145]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

- Na linha 4, você passa o query para cursor.execute().

- Agora você pode criar sua userstabela usando esta função:

In [146]:
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INT AUTO_INCREMENT, 
  name TEXT NOT NULL, 
  age INT, 
  gender TEXT, 
  nationality TEXT, 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""


execute_query(connection, create_users_table)


Query executed successfully


- A consulta para implementar a relação de chave estrangeira é um pouco diferente no MySQL em comparação com o SQLite. 
- Além disso, o MySQL usa a AUTO_INCREMENT palavra - chave (em comparação com a AUTOINCREMENT palavra-chave SQLite ) para criar colunas onde os valores são incrementados automaticamente quando novos registros são inseridos.

- O script a seguir cria a posts tabela, que contém uma chave estrangeira user_id que faz referência à coluna id da tabela users:

In [147]:
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts (
  id INT AUTO_INCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY fk_user_id (user_id) REFERENCES users(id), 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""



execute_query(connection, create_posts_table)

Query executed successfully


In [155]:
create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INT AUTO_INCREMENT,
  user_id INTEGER NOT NULL,
  post_id INTEGER NOT NULL,
  FOREIGN KEY fk_user_id (user_id) REFERENCES users(id),
  FOREIGN KEY fk_post_id (post_id) REFERENCES posts(id), 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""

create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INT AUTO_INCREMENT,
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL,
  FOREIGN KEY fk_user_id (user_id) REFERENCES users(id), 
  FOREIGN KEY fk_post_id (post_id) REFERENCES posts(id), 
  PRIMARY KEY (id)
) ENGINE = InnoDB
"""
execute_query(connection, create_likes_table)
execute_query(connection, create_comments_table)

Query executed successfully
Query executed successfully


### Inserindo Registros
- Existem duas maneiras de inserir registros nos bancos de dados MySQL a partir de um aplicativo Python. 
    - A primeira abordagem é semelhante ao SQLite. Você pode armazenar a INSERT INTO consulta em uma sequência e, em seguida, usar cursor.execute()para inserir registros.

- Anteriormente, você definiu uma função de wrapper execute_query() usada para inserir registros. Você pode usar esta mesma função agora para inserir registros na sua tabela MySQL. O script a seguir insere registros na tabela users usando execute_query():

In [149]:
create_users = """
INSERT INTO
  `users` (`name`, `age`, `gender`, `nationality`)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

execute_query(connection, create_users)  

create_posts = """
INSERT INTO
  `posts` (`title`, `description`, `user_id`)
VALUES
  ('Happy', 'I am feeling very happy today', 1),
  ('Hot Weather', 'The weather is very hot today', 2),
  ('Help', 'I need some help with my work', 2),
  ('Great News', 'I am getting married', 1),
  ('Interesting Game', 'It was a fantastic game of tennis', 5),
  ('Party', 'Anyone up for a late-night party today?', 3);
"""

execute_query(connection, create_posts) 



Query executed successfully
Query executed successfully


In [156]:
create_comments = """
INSERT INTO
  `comments` (`text`, `user_id`, `post_id`)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

execute_query(connection, create_comments)

Query executed successfully


- A segunda abordagem utiliza cursor.executemany(), que aceita dois parâmetros:

    - A sequência de consultas que contém espaços reservados para os registros a serem inseridos
    - A lista de registros que você deseja inserir
- Veja o exemplo a seguir, que insere dois registros na tabela likes:

In [107]:
sql = "INSERT INTO likes ( user_id, post_id ) VALUES ( %s, %s )"
val = [(4, 5), (3, 4)]

cursor = connection.cursor()
cursor.executemany(sql, val)
connection.commit()

In [151]:

create_likes = """
INSERT INTO
  `likes` (`user_id`, `post_id`)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

execute_query(connection, create_likes) 

Query executed successfully


### Selecionando Registros

- O processo de seleção de registros no MySQL é absolutamente idêntico ao da seleção de registros no SQLite. Você pode usar cursor.execute()seguido por .fetchall(). O script a seguir cria uma função de wrapper execute_read_query()que você pode usar para selecionar registros:

In [152]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [159]:
select_users = "SELECT * FROM users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)

(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')


In [153]:
select_posts = "SELECT * FROM posts"
posts = execute_read_query(connection, select_posts)

for post in posts:
    print(post)

(1, 'Happy', 'I am feeling very happy today', 1)
(2, 'Hot Weather', 'The weather is very hot today', 2)
(3, 'Help', 'I need some help with my work', 2)
(4, 'Great News', 'I am getting married', 1)
(5, 'Interesting Game', 'It was a fantastic game of tennis', 5)
(6, 'Party', 'Anyone up for a late-night party today?', 3)


In [157]:
select_comments = "SELECT * FROM comments"
comments = execute_read_query(connection, select_comments)

for comment in comments:
    print(comment)

(1, 'Count me in', 1, 6)
(2, 'What sort of help?', 5, 3)
(3, 'Congrats buddy', 2, 4)
(4, 'I was rooting for Nadal though', 4, 5)
(5, 'Help with your thesis?', 2, 3)
(6, 'Many congratulations', 5, 4)


In [158]:
select_likes = "SELECT * FROM likes"
likes = execute_read_query(connection, select_likes)

for like in likes:
    print(like)

(1, 1, 6)
(2, 2, 3)
(3, 1, 5)
(4, 5, 4)
(5, 2, 4)
(6, 4, 2)
(7, 3, 6)


### Atualizando registro da tabela

- O processo de atualização de registros no MySQL mysql-connector-python também é uma cópia do sqlite3 módulo Python SQL. Você precisa passar a consulta de string para cursor.execute(). Por exemplo, o script a seguir atualiza a descrição da postagem com um idde 2:

In [160]:
update_post_description = """
UPDATE
  posts
SET
  description = "The weather has become pleasant now"
WHERE
  id = 2
"""

execute_query(connection,  update_post_description)

Query executed successfully


### Excluindo registro da tabela
- O processo de exclusão no MySQL também é semelhante ao SQLite, conforme mostrado no exemplo a seguir:

In [161]:
delete_comment = "DELETE FROM comments WHERE id = 2"
execute_query(connection, delete_comment)

Query executed successfully


In [162]:
select_comments = "SELECT * FROM comments"
comments = execute_read_query(connection, select_comments)

for comment in comments:
    print(comment)

(1, 'Count me in', 1, 6)
(3, 'Congrats buddy', 2, 4)
(4, 'I was rooting for Nadal though', 4, 5)
(5, 'Help with your thesis?', 2, 3)
(6, 'Many congratulations', 5, 4)
